In [1]:
import requests
import matplotlib.pyplot as plt
import pandas as pd
import json
from datetime import datetime

# Tu API key de OpenWeatherMap
API_KEY = "2aacbfc0427f4a869195e14b981a7508"

def obtener_datos_clima(ciudad, pais=''):
    """
    Obtiene datos del clima de la API de OpenWeatherMap
    """
    if pais:
        ubicacion = f"{ciudad},{pais}"
    else:
        ubicacion = ciudad

    url = f"http://api.openweathermap.org/data/2.5/weather?q={ubicacion}&APPID={API_KEY}&units=metric"

    try:
        respuesta = requests.get(url)
        respuesta.raise_for_status()  # Lanza excepción si hay error HTTP

        datos = respuesta.json()
        return datos
    except requests.exceptions.RequestException as e:
        print(f"Error al conectar con la API: {e}")
        return None
    except json.JSONDecodeError as e:
        print(f"Error al decodificar JSON: {e}")
        return None

def procesar_datos_clima(datos):
    """
    Procesa los datos del clima para hacerlos más legibles
    """
    if datos is None:
        return None

    # Extraer información relevante
    info_clima = {
        'ciudad': datos['name'],
        'pais': datos['sys']['country'],
        'temperatura': datos['main']['temp'],
        'sensacion_termica': datos['main']['feels_like'],
        'temperatura_min': datos['main']['temp_min'],
        'temperatura_max': datos['main']['temp_max'],
        'humedad': datos['main']['humidity'],
        'presion': datos['main']['pressure'],
        'descripcion': datos['weather'][0]['description'],
        'viento_velocidad': datos['wind']['speed'],
        'nubosidad': datos['clouds']['all'],
        'fecha_consulta': datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    }

    return info_clima

def graficar_datos_clima(datos):
    """
    Crea gráficas con los datos del clima
    """
    if datos is None:
        print("No hay datos para graficar")
        return

    # Crear figura con subgráficas
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 10))
    fig.suptitle(f'Datos del Clima - {datos["ciudad"]}, {datos["pais"]}', fontsize=16, fontweight='bold')

    # Gráfica 1: Temperaturas
    temperaturas = ['Actual', 'Sensación', 'Mínima', 'Máxima']
    valores_temp = [datos['temperatura'], datos['sensacion_termica'],
                   datos['temperatura_min'], datos['temperatura_max']]
    colores_temp = ['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4']

    bars = ax1.bar(temperaturas, valores_temp, color=colores_temp, alpha=0.7)
    ax1.set_title('Temperaturas (°C)')
    ax1.set_ylabel('Temperatura (°C)')

    # Añadir valores en las barras
    for bar, valor in zip(bars, valores_temp):
        ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.1,
                f'{valor:.1f}°C', ha='center', va='bottom', fontweight='bold')

    # Gráfica 2: Humedad y Presión
    categorias = ['Humedad', 'Presión']
    valores = [datos['humedad'], datos['presion']/10]  # Dividir presión por 10 para escala
    colores = ['#A3DE83', '#FE9C45']

    bars2 = ax2.bar(categorias, valores, color=colores, alpha=0.7)
    ax2.set_title('Humedad y Presión')
    ax2.set_ylabel('Humedad (%) / Presión (hPa)')

    # Añadir valores en las barras
    for bar, valor, cat in zip(bars2, valores, categorias):
        if cat == 'Humedad':
            texto = f'{valor:.0f}%'
        else:
            texto = f'{valor:.0f} hPa'
        ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1,
                texto, ha='center', va='bottom', fontweight='bold')

    # Gráfica 3: Velocidad del Viento
    categorias_viento = ['Velocidad del Viento']
    valores_viento = [datos['viento_velocidad']]

    ax3.bar(categorias_viento, valores_viento, color='#9B59B6', alpha=0.7)
    ax3.set_title('Velocidad del Viento')
    ax3.set_ylabel('Velocidad (m/s)')
    ax3.text(0, datos['viento_velocidad'] + 0.1, f'{datos["viento_velocidad"]} m/s',
             ha='center', va='bottom', fontweight='bold')

    # Gráfica 4: Nubosidad
    categorias_nubes = ['Nubosidad']
    valores_nubes = [datos['nubosidad']]

    ax4.bar(categorias_nubes, valores_nubes, color='#3498DB', alpha=0.7)
    ax4.set_title('Nubosidad')
    ax4.set_ylabel('Nubosidad (%)')
    ax4.text(0, datos['nubosidad'] + 1, f'{datos["nubosidad"]}%',
             ha='center', va='bottom', fontweight='bold')

    plt.tight_layout()
    plt.show()

    # Mostrar información adicional
    print(f"\n📊 RESUMEN DEL CLIMA EN {datos['ciudad'].upper()}, {datos['pais']}")
    print("=" * 50)
    print(f"🌡️  Temperatura actual: {datos['temperatura']:.1f}°C")
    print(f"🌡️  Sensación térmica: {datos['sensacion_termica']:.1f}°C")
    print(f"📈 Temperatura máxima: {datos['temperatura_max']:.1f}°C")
    print(f"📉 Temperatura mínima: {datos['temperatura_min']:.1f}°C")
    print(f"💧 Humedad: {datos['humedad']}%")
    print(f"🌬️  Velocidad del viento: {datos['viento_velocidad']} m/s")
    print(f"☁️  Nubosidad: {datos['nubosidad']}%")
    print(f"📝 Descripción: {datos['descripcion'].title()}")
    print(f"🕐 Consulta realizada: {datos['fecha_consulta']}")

# Ejemplo de uso
def main():
    # Solicitar ciudad al usuario
    ciudad = input("Ingresa el nombre de la ciudad: ").strip()
    pais = input("Ingresa el código del país (opcional, ej: us, mx, es): ").strip()

    print(f"\n🔍 Buscando datos del clima para {ciudad}...")

    # Obtener datos
    datos_raw = obtener_datos_clima(ciudad, pais)

    if datos_raw and datos_raw.get('cod') == 200:
        datos_procesados = procesar_datos_clima(datos_raw)
        graficar_datos_clima(datos_procesados)
    else:
        print("❌ Error: No se pudieron obtener los datos del clima")
        if datos_raw:
            print(f"Código de error: {datos_raw.get('cod')}")
            print(f"Mensaje: {datos_raw.get('message', 'Error desconocido')}")

# Ejecutar el programa
if __name__ == "__main__":
    main()


🔍 Buscando datos del clima para Bogota...
Error al conectar con la API: 401 Client Error: Unauthorized for url: http://api.openweathermap.org/data/2.5/weather?q=Bogota,bog&APPID=2aacbfc0427f4a869195e14b981a7508&units=metric
❌ Error: No se pudieron obtener los datos del clima
